# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df = weather_df.drop(weather_df.columns[0], axis=1)

weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,Dunedin,54,NZ,1603993480,93,-45.87,170.50,53.01,8.01
1,Butaritari,68,KI,1603993480,77,3.07,172.79,81.73,12.57
2,Bluff,86,NZ,1603992926,91,-46.60,168.33,52.00,3.00
3,Castro,100,BR,1603993480,59,-24.79,-50.01,71.15,12.06
4,Ndélé,88,CF,1603993480,53,8.41,20.65,78.35,2.08
...,...,...,...,...,...,...,...,...,...
577,Halifax,100,CA,1603993531,73,44.65,-63.57,50.00,1.99
578,Sydney,0,AU,1603993370,87,-33.87,151.21,57.20,3.20
579,Mozelos,0,PT,1603993611,87,41.87,-8.40,61.00,4.70
580,Clyde River,100,CA,1603993611,95,70.47,-68.59,20.46,9.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_df[["Lat", "Long"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [4]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = weather_df.loc[(weather_df['Max Temp'] >= 70) & (weather_df['Max Temp'] <= 80) & (weather_df['Cloudiness']==0) & (weather_df['Wind Speed'] < 10)]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
20,Angoche,0,MZ,1603993482,77,-16.23,39.91,78.01,6.76
151,Sakakah,0,SA,1603993515,12,29.97,40.21,77.00,9.17
251,General Roca,0,AR,1603993540,19,-39.03,-67.58,71.60,6.93
305,Buraidah,0,SA,1603993555,22,26.33,43.98,75.20,5.82
503,Qeshm,0,IR,1603993597,19,26.96,56.27,75.20,6.93
528,Arāria,0,IN,1603993602,60,26.15,87.52,75.83,1.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df["Hotel Name"]=""
hotel_df 

C:\Users\vivie\anaconda3\envs\PythonData1\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
20,Angoche,0,MZ,1603993482,77,-16.23,39.91,78.01,6.76,
151,Sakakah,0,SA,1603993515,12,29.97,40.21,77.00,9.17,
251,General Roca,0,AR,1603993540,19,-39.03,-67.58,71.60,6.93,
305,Buraidah,0,SA,1603993555,22,26.33,43.98,75.20,5.82,
503,Qeshm,0,IR,1603993597,19,26.96,56.27,75.20,6.93,
528,Arāria,0,IN,1603993602,60,26.15,87.52,75.83,1.50,


In [8]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key,
    "location": "23.61,58.59"
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
name_address = requests.get(base_url, params=params)
    
name_address = name_address.json()

name_address

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 23.5671512, 'lng': 58.6155072},
    'viewport': {'northeast': {'lat': 23.56829382989272,
      'lng': 58.61675992989272},
     'southwest': {'lat': 23.56559417010727, 'lng': 58.61406027010727}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Al Bustan Palace, A Ritz-Carlton Hotel',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 576,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108593968168219378701">Al Bustan Palace, a Ritz-Carlton Hotel</a>'],
     'photo_reference': 'CmRaAAAAeCyWs2dQvmNiN5EN7c3woJlmpHfKuAnvWhpdLg43lh42qN_YGkZxFGUS4b4vtk3-th5U2_G7FN74s_u-SGy8ja7Wlbpcmbv-F0MxgW9UHeI3YVR2erKfQNY4f-OpD6ULEhCJRJOeng8uMUDjRC3tYejQGhSxy9cDO_4K14OMrROeLPwnCx69PQ',
     'width': 1024}],
   'place_id': 'ChIJgUYW6zrxkT4RIaJvt4wvA_Q',
   'plus_code': {'compound_code': 'HJ88+V6 Muscat, Om

In [11]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Long"] 
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_result = requests.get(base_url, params=params)
    
    hotel_result = hotel_result.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_result["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\vivie\anaconda3\envs\PythonData1\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [12]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
20,Angoche,0,MZ,1603993482,77,-16.23,39.91,78.01,6.76,Guest House HESADA
151,Sakakah,0,SA,1603993515,12,29.97,40.21,77.00,9.17,Alnuzl Special Hotel
251,General Roca,0,AR,1603993540,19,-39.03,-67.58,71.60,6.93,Hotel y Casino del Río
305,Buraidah,0,SA,1603993555,22,26.33,43.98,75.20,5.82,فندق براند القصيم
503,Qeshm,0,IR,1603993597,19,26.96,56.27,75.20,6.93,Irman BOUTIQUE HOTEL
528,Arāria,0,IN,1603993602,60,26.15,87.52,75.83,1.50,Hotel Ever Green Araria


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [18]:
# Add marker layer ontop of heat map

locations = weather_df[["Lat", "Long"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

hotel_locations = hotel_df[["Lat", "Long"]].astype(float)

markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))